In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv('/kaggle/input/icc-mens-world-cup-2023/deliveries.csv')

In [3]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,JM Bairstow,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.isnull().sum()

match_id                      0
season                        0
start_date                    0
venue                         0
innings                       0
ball                          0
batting_team                  0
bowling_team                  0
striker                       0
non_striker                   0
bowler                        0
runs_off_bat                  0
extras                        0
wides                     24866
noballs                   25459
byes                      25476
legbyes                   25325
penalty                   25513
wicket_type               24799
player_dismissed          24799
other_wicket_type         25513
other_player_dismissed    25513
dtype: int64

In [5]:
df=df.drop(['season','non_striker','start_date','striker','wides','noballs','byes','legbyes','penalty','player_dismissed','other_player_dismissed','other_wicket_type'],axis=1)

In [6]:
df['cumulative_runs'] = df.groupby(['match_id', 'innings'])['runs_off_bat'].cumsum() + df.groupby(['match_id', 'innings'])['extras'].cumsum()

In [7]:
df['runs'] = df.groupby(['match_id', 'innings','bowler'])['runs_off_bat'].cumsum() + df.groupby(['match_id', 'innings'])['extras'].cumsum()

In [8]:
df=df.drop(['innings','runs_off_bat','extras'],axis=1)

In [9]:
df[(df['bowler'] == 'TA Boult') & (df['batting_team'] == 'England') & (df['wicket_type'].notna())]

,match_id,venue,ball,batting_team,bowling_team,bowler,wicket_type,cumulative_runs,runs
235,1,"Narendra Modi Stadium, Ahmedabad",38.5,England,New Zealand,TA Boult,caught,221,41


In [10]:
df['wicket']=0

In [11]:
df

,match_id,venue,ball,batting_team,bowling_team,bowler,wicket_type,cumulative_runs,runs,wicket
0,1,"Narendra Modi Stadium, Ahmedabad",0.1,England,New Zealand,TA Boult,NaN,0,0,0
1,1,"Narendra Modi Stadium, Ahmedabad",0.2,England,New Zealand,TA Boult,NaN,6,6,0
2,1,"Narendra Modi Stadium, Ahmedabad",0.3,England,New Zealand,TA Boult,NaN,7,7,0
3,1,"Narendra Modi Stadium, Ahmedabad",0.4,England,New Zealand,TA Boult,NaN,8,8,0
4,1,"Narendra Modi Stadium, Ahmedabad",0.5,England,New Zealand,TA Boult,NaN,12,12,0
...,...,...,...,...,...,...,...,...,...,...
25509,48,"Narendra Modi Stadium, Ahmedabad",42.2,Australia,India,Mohammed Siraj,NaN,237,59,0
25510,48,"Narendra Modi Stadium, Ahmedabad",42.3,Australia,India,Mohammed Siraj,NaN,238,60,0
25511,48,"Narendra Modi Stadium, Ahmedabad",42.4,Australia,India,Mohammed Siraj,NaN,239,61,0
25512,48,"Narendra Modi Stadium, Ahmedabad",42.5,Australia,India,Mohammed Siraj,caught,239,61,0


In [12]:
df.loc[df['wicket_type'].notna(), 'wicket'] = 1

In [13]:
df

,match_id,venue,ball,batting_team,bowling_team,bowler,wicket_type,cumulative_runs,runs,wicket
0,1,"Narendra Modi Stadium, Ahmedabad",0.1,England,New Zealand,TA Boult,NaN,0,0,0
1,1,"Narendra Modi Stadium, Ahmedabad",0.2,England,New Zealand,TA Boult,NaN,6,6,0
2,1,"Narendra Modi Stadium, Ahmedabad",0.3,England,New Zealand,TA Boult,NaN,7,7,0
3,1,"Narendra Modi Stadium, Ahmedabad",0.4,England,New Zealand,TA Boult,NaN,8,8,0
4,1,"Narendra Modi Stadium, Ahmedabad",0.5,England,New Zealand,TA Boult,NaN,12,12,0
...,...,...,...,...,...,...,...,...,...,...
25509,48,"Narendra Modi Stadium, Ahmedabad",42.2,Australia,India,Mohammed Siraj,NaN,237,59,0
25510,48,"Narendra Modi Stadium, Ahmedabad",42.3,Australia,India,Mohammed Siraj,NaN,238,60,0
25511,48,"Narendra Modi Stadium, Ahmedabad",42.4,Australia,India,Mohammed Siraj,NaN,239,61,0
25512,48,"Narendra Modi Stadium, Ahmedabad",42.5,Australia,India,Mohammed Siraj,caught,239,61,1


In [14]:
df=df.drop(['wicket_type'],axis=1)

In [15]:
df

,match_id,venue,ball,batting_team,bowling_team,bowler,cumulative_runs,runs,wicket
0,1,"Narendra Modi Stadium, Ahmedabad",0.1,England,New Zealand,TA Boult,0,0,0
1,1,"Narendra Modi Stadium, Ahmedabad",0.2,England,New Zealand,TA Boult,6,6,0
2,1,"Narendra Modi Stadium, Ahmedabad",0.3,England,New Zealand,TA Boult,7,7,0
3,1,"Narendra Modi Stadium, Ahmedabad",0.4,England,New Zealand,TA Boult,8,8,0
4,1,"Narendra Modi Stadium, Ahmedabad",0.5,England,New Zealand,TA Boult,12,12,0
...,...,...,...,...,...,...,...,...,...
25509,48,"Narendra Modi Stadium, Ahmedabad",42.2,Australia,India,Mohammed Siraj,237,59,0
25510,48,"Narendra Modi Stadium, Ahmedabad",42.3,Australia,India,Mohammed Siraj,238,60,0
25511,48,"Narendra Modi Stadium, Ahmedabad",42.4,Australia,India,Mohammed Siraj,239,61,0
25512,48,"Narendra Modi Stadium, Ahmedabad",42.5,Australia,India,Mohammed Siraj,239,61,1


In [16]:
df['wicket'] = df.groupby(['match_id', 'bowler'])['wicket'].transform('sum')

In [17]:
df = df.drop_duplicates()

In [18]:
df

,match_id,venue,ball,batting_team,bowling_team,bowler,cumulative_runs,runs,wicket
0,1,"Narendra Modi Stadium, Ahmedabad",0.1,England,New Zealand,TA Boult,0,0,1
1,1,"Narendra Modi Stadium, Ahmedabad",0.2,England,New Zealand,TA Boult,6,6,1
2,1,"Narendra Modi Stadium, Ahmedabad",0.3,England,New Zealand,TA Boult,7,7,1
3,1,"Narendra Modi Stadium, Ahmedabad",0.4,England,New Zealand,TA Boult,8,8,1
4,1,"Narendra Modi Stadium, Ahmedabad",0.5,England,New Zealand,TA Boult,12,12,1
...,...,...,...,...,...,...,...,...,...
25509,48,"Narendra Modi Stadium, Ahmedabad",42.2,Australia,India,Mohammed Siraj,237,59,1
25510,48,"Narendra Modi Stadium, Ahmedabad",42.3,Australia,India,Mohammed Siraj,238,60,1
25511,48,"Narendra Modi Stadium, Ahmedabad",42.4,Australia,India,Mohammed Siraj,239,61,1
25512,48,"Narendra Modi Stadium, Ahmedabad",42.5,Australia,India,Mohammed Siraj,239,61,1


In [19]:
df=df.drop(['match_id'],axis=1)

In [20]:
df[(df['bowling_team'] == 'New Zealand') & (df['batting_team'] == 'England')]

,venue,ball,batting_team,bowling_team,bowler,cumulative_runs,runs,wicket
0,"Narendra Modi Stadium, Ahmedabad",0.1,England,New Zealand,TA Boult,0,0,1
1,"Narendra Modi Stadium, Ahmedabad",0.2,England,New Zealand,TA Boult,6,6,1
2,"Narendra Modi Stadium, Ahmedabad",0.3,England,New Zealand,TA Boult,7,7,1
3,"Narendra Modi Stadium, Ahmedabad",0.4,England,New Zealand,TA Boult,8,8,1
4,"Narendra Modi Stadium, Ahmedabad",0.5,England,New Zealand,TA Boult,12,12,1
...,...,...,...,...,...,...,...,...
299,"Narendra Modi Stadium, Ahmedabad",49.3,England,New Zealand,JDS Neesham,278,54,0
300,"Narendra Modi Stadium, Ahmedabad",49.4,England,New Zealand,JDS Neesham,279,55,0
301,"Narendra Modi Stadium, Ahmedabad",49.5,England,New Zealand,JDS Neesham,280,56,0
302,"Narendra Modi Stadium, Ahmedabad",49.6,England,New Zealand,JDS Neesham,281,57,0


In [21]:
df['bowler'].unique()

array(['TA Boult', 'MJ Henry', 'MJ Santner', 'JDS Neesham', 'R Ravindra',
       'GD Phillips', 'CR Woakes', 'SM Curran', 'MA Wood', 'MM Ali',
       'AU Rashid', 'LS Livingstone', 'A Dutt', 'LV van Beek',
       'CN Ackermann', 'PA van Meekeren', 'BFW de Leede',
       'RE van der Merwe', 'Vikramjit Singh', 'Saqib Zulfiqar',
       'Shaheen Shah Afridi', 'Hasan Ali', 'Haris Rauf', 'Iftikhar Ahmed',
       'Mohammad Nawaz', 'Shadab Khan', 'Taskin Ahmed', 'Shoriful Islam',
       'Mustafizur Rahman', 'Shakib Al Hasan', 'Mehedi Hasan Miraz',
       'Mahmudullah', 'Fazalhaq Farooqi', 'Mujeeb Ur Rahman',
       'Naveen-ul-Haq', 'Rashid Khan', 'Mohammad Nabi',
       'Azmatullah Omarzai', 'CAK Rajitha', 'D Madushanka', 'MD Shanaka',
       'DM de Silva', 'M Pathirana', 'DN Wellalage', 'L Ngidi',
       'M Jansen', 'K Rabada', 'KA Maharaj', 'G Coetzee', 'JJ Bumrah',
       'Mohammed Siraj', 'HH Pandya', 'R Ashwin', 'Kuldeep Yadav',
       'RA Jadeja', 'MA Starc', 'JR Hazlewood', 'PJ Cummins'

In [22]:
bowler_dict = df['bowler'].value_counts().to_dict()
i=1
for bowler in bowler_dict:
    bowler_dict[bowler]=i
    i+=1
bowler_dict

{'A Zampa': 1,
 'JR Hazlewood': 2,
 'TA Boult': 3,
 'MA Starc': 4,
 'MJ Santner': 5,
 'PJ Cummins': 6,
 'KA Maharaj': 7,
 'Rashid Khan': 8,
 'JJ Bumrah': 9,
 'Kuldeep Yadav': 10,
 'RA Jadeja': 11,
 'Shaheen Shah Afridi': 12,
 'Mujeeb Ur Rahman': 13,
 'D Madushanka': 14,
 'Haris Rauf': 15,
 'AU Rashid': 16,
 'Mohammed Siraj': 17,
 'M Jansen': 18,
 'K Rabada': 19,
 'M Theekshana': 20,
 'Mehedi Hasan Miraz': 21,
 'A Dutt': 22,
 'GJ Maxwell': 23,
 'Mustafizur Rahman': 24,
 'PA van Meekeren': 25,
 'Shoriful Islam': 26,
 'R Ravindra': 27,
 'G Coetzee': 28,
 'LV van Beek': 29,
 'L Ngidi': 30,
 'Shakib Al Hasan': 31,
 'Mohammad Nabi': 32,
 'BFW de Leede': 33,
 'Taskin Ahmed': 34,
 'CR Woakes': 35,
 'MA Wood': 36,
 'Naveen-ul-Haq': 37,
 'LH Ferguson': 38,
 'MJ Henry': 39,
 'Hasan Ali': 40,
 'DJ Willey': 41,
 'CAK Rajitha': 42,
 'Iftikhar Ahmed': 43,
 'RE van der Merwe': 44,
 'CN Ackermann': 45,
 'Mohammed Shami': 46,
 'MM Ali': 47,
 'Fazalhaq Farooqi': 48,
 'GD Phillips': 49,
 'T Shamsi': 50,
 

In [23]:
df['bowler'] = df['bowler'].map(bowler_dict)

In [24]:
batting_dict = df['batting_team'].value_counts().to_dict()
i=1
for bat in batting_dict:
    batting_dict[bat]=i
    i+=1
batting_dict

{'Australia': 1,
 'South Africa': 2,
 'India': 3,
 'New Zealand': 4,
 'Bangladesh': 5,
 'Afghanistan': 6,
 'Pakistan': 7,
 'England': 8,
 'Sri Lanka': 9,
 'Netherlands': 10}

In [25]:
df['batting_team'] = df['batting_team'].map(batting_dict)
df['bowling_team'] = df['bowling_team'].map(batting_dict)

In [26]:
venue_dict = df['venue'].value_counts().to_dict()
i=1
for van in venue_dict:
    venue_dict[van]=i
    i+=1
venue_dict

{'MA Chidambaram Stadium, Chepauk, Chennai': 1,
 'Himachal Pradesh Cricket Association Stadium, Dharamsala': 2,
 'Maharashtra Cricket Association Stadium, Pune': 3,
 'Narendra Modi Stadium, Ahmedabad': 4,
 'Eden Gardens, Kolkata': 5,
 'Wankhede Stadium, Mumbai': 6,
 'Arun Jaitley Stadium, Delhi': 7,
 'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow': 8,
 'M Chinnaswamy Stadium, Bengaluru': 9,
 'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 10}

In [27]:
df['venue'] = df['venue'].map(venue_dict)

In [28]:
#df = pd.concat([df, df], ignore_index=True)

In [29]:
#df = pd.concat([df, df], ignore_index=True)

In [30]:
df.reset_index(drop=True, inplace=True)

In [31]:
additional_columns = df.drop('wicket', axis=1)

# Perform one-hot encoding for the 'wicket' column
df_encoded = pd.get_dummies(df, columns=['wicket'], prefix='wicket')

# Convert one-hot encoded columns to binary format (1, 0)
for column in df_encoded.columns:
    if 'wicket' in column:
        df_encoded[column] = df_encoded[column].apply(lambda x: 1 if x > 0 else 0)

# Concatenate the one-hot encoded DataFrame with additional columns
df = df_encoded

In [32]:
df["wicket_7"]='0'
df["wicket_8"]='0'
df["wicket_9"]='0'
df["wicket_10"]='0'

In [33]:
df

,venue,ball,batting_team,bowling_team,bowler,cumulative_runs,runs,wicket_0,wicket_1,wicket_2,wicket_3,wicket_4,wicket_5,wicket_6,wicket_7,wicket_8,wicket_9,wicket_10
0,4,0.1,8,4,3,0,0,0,1,0,0,0,0,0,0,0,0,0
1,4,0.2,8,4,3,6,6,0,1,0,0,0,0,0,0,0,0,0
2,4,0.3,8,4,3,7,7,0,1,0,0,0,0,0,0,0,0,0
3,4,0.4,8,4,3,8,8,0,1,0,0,0,0,0,0,0,0,0
4,4,0.5,8,4,3,12,12,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25509,4,42.2,1,3,17,237,59,0,1,0,0,0,0,0,0,0,0,0
25510,4,42.3,1,3,17,238,60,0,1,0,0,0,0,0,0,0,0,0
25511,4,42.4,1,3,17,239,61,0,1,0,0,0,0,0,0,0,0,0
25512,4,42.5,1,3,17,239,61,0,1,0,0,0,0,0,0,0,0,0


In [34]:
df.dtypes

venue                int64
ball               float64
batting_team         int64
bowling_team         int64
bowler               int64
cumulative_runs      int64
runs                 int64
wicket_0             int64
wicket_1             int64
wicket_2             int64
wicket_3             int64
wicket_4             int64
wicket_5             int64
wicket_6             int64
wicket_7            object
wicket_8            object
wicket_9            object
wicket_10           object
dtype: object

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [36]:
# Select features (X) by excluding the target variable columns
X = df.drop(['wicket_0','wicket_1', 'wicket_2', 'wicket_3', 'wicket_4', 'wicket_5', 'wicket_6', 'wicket_7', 'wicket_8', 'wicket_9', 'wicket_10'], axis=1)

# Select the target variable (y) by concatenating the one-hot encoded columns
y = df[['wicket_0','wicket_1', 'wicket_2', 'wicket_3', 'wicket_4', 'wicket_5', 'wicket_6', 'wicket_7', 'wicket_8', 'wicket_9', 'wicket_10']]


In [37]:
X

,venue,ball,batting_team,bowling_team,bowler,cumulative_runs,runs
0,4,0.1,8,4,3,0,0
1,4,0.2,8,4,3,6,6
2,4,0.3,8,4,3,7,7
3,4,0.4,8,4,3,8,8
4,4,0.5,8,4,3,12,12
...,...,...,...,...,...,...,...
25509,4,42.2,1,3,17,237,59
25510,4,42.3,1,3,17,238,60
25511,4,42.4,1,3,17,239,61
25512,4,42.5,1,3,17,239,61


In [38]:
y

,wicket_0,wicket_1,wicket_2,wicket_3,wicket_4,wicket_5,wicket_6,wicket_7,wicket_8,wicket_9,wicket_10
0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
25509,0,1,0,0,0,0,0,0,0,0,0
25510,0,1,0,0,0,0,0,0,0,0,0
25511,0,1,0,0,0,0,0,0,0,0,0
25512,0,1,0,0,0,0,0,0,0,0,0


In [39]:
X = X.astype('float32')
y = y.astype('float32')

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [42]:
model = Sequential()
model.add(Dense(4, activation='relu', input_dim=X_train.shape[1]))
model.add(BatchNormalization())
model.add(Dense(6, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
model.fit(X_train, y_train, epochs=100 , batch_size=32, validation_split=0.2)

Epoch 1/100
511/511 [==============================] - 2s 4ms/step - loss: 0.2121 - accuracy: 0.3531 - val_loss: 0.2092 - val_accuracy: 0.3877
Epoch 2/100
511/511 [==============================] - 2s 3ms/step - loss: 0.2118 - accuracy: 0.3471 - val_loss: 0.2091 - val_accuracy: 0.3887
Epoch 3/100
511/511 [==============================] - 2s 4ms/step - loss: 0.2122 - accuracy: 0.3474 - val_loss: 0.2095 - val_accuracy: 0.3835
Epoch 4/100
511/511 [==============================] - 2s 4ms/step - loss: 0.2124 - accuracy: 0.3477 - val_loss: 0.2094 - val_accuracy: 0.3794
Epoch 5/100
511/511 [==============================] - 2s 3ms/step - loss: 0.2125 - accuracy: 0.3482 - val_loss: 0.2094 - val_accuracy: 0.3845
Epoch 6/100
511/511 [==============================] - 2s 3ms/step - loss: 0.2121 - accuracy: 0.3477 - val_loss: 0.2094 - val_accuracy: 0.3767
Epoch 7/100
511/511 [==============================] - 2s 3ms/step - loss: 0.2123 - accuracy: 0.3594 - val_loss: 0.2089 - val_accuracy: 0.3877

In [47]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

160/160 [==============================] - 0s 2ms/step - loss: 0.2096 - accuracy: 0.3815
Test Accuracy: 0.3815402686595917


In [48]:
y_pred = model.predict(X_test)

160/160 [==============================] - 0s 2ms/step


In [49]:
pd.DataFrame(y_pred)

,0,1,2,3,4,5,6,7,8,9,10
0,0.081619,0.232041,0.538894,0.134653,0.006641,0.003661,0.002491,6.236506e-11,3.249756e-11,6.250248e-11,1.454443e-11
1,0.207448,0.283750,0.315148,0.136895,0.036359,0.020091,0.000309,9.225773e-11,7.091450e-11,5.641871e-11,7.063614e-11
2,0.369262,0.294343,0.206325,0.097291,0.028343,0.002023,0.002414,2.314025e-10,3.410104e-10,2.737259e-10,4.407821e-10
3,0.200539,0.525963,0.173585,0.082133,0.016816,0.000856,0.000108,3.086474e-13,3.488432e-13,1.957457e-13,1.976599e-12
4,0.235543,0.367943,0.248335,0.112450,0.029781,0.005515,0.000434,3.401650e-11,3.298074e-11,2.442819e-11,6.656983e-11
...,...,...,...,...,...,...,...,...,...,...,...
5098,0.253065,0.294943,0.277057,0.123249,0.038314,0.012316,0.001056,4.804783e-10,4.469291e-10,3.872426e-10,5.582796e-10
5099,0.305941,0.298943,0.237502,0.108682,0.038691,0.006040,0.004201,2.607579e-09,3.123716e-09,2.902835e-09,4.569885e-09
5100,0.450508,0.277094,0.169761,0.083386,0.017699,0.000395,0.001156,7.832473e-12,1.587436e-11,1.017052e-11,1.536586e-11
5101,0.307835,0.300332,0.234220,0.106987,0.039612,0.006975,0.004038,2.930573e-09,3.369211e-09,3.209465e-09,5.411968e-09


In [50]:
def max_to_one(row):
    max_value = row.max()
    return row.apply(lambda x: 1 if x == max_value else 0)

# Apply the function to each row
y_pred_df = pd.DataFrame(y_pred).apply(max_to_one, axis=1)

In [51]:
y_pred_df

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5098,0,1,0,0,0,0,0,0,0,0,0
5099,1,0,0,0,0,0,0,0,0,0,0
5100,1,0,0,0,0,0,0,0,0,0,0
5101,1,0,0,0,0,0,0,0,0,0,0


In [52]:
y_test

,wicket_0,wicket_1,wicket_2,wicket_3,wicket_4,wicket_5,wicket_6,wicket_7,wicket_8,wicket_9,wicket_10
10274,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7330,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11487,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25066,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24841,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
12906,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23449,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17581,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9929,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [54]:
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f'R-squared (R2) Score: {r2}')

R-squared (R2) Score: 0.37538024413830867


In [55]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [56]:
# Create a decision tree regressor
model = DecisionTreeRegressor()

# Define the hyperparameter grid
param_grid = {
    'max_depth': [ 1,3,4],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create GridSearchCV object with 'r2' scoring
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f'Best Hyperparameters: {best_params}')

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate R2 score for the best model
r2 = r2_score(y_test, y_pred)
print(f'R-squared (R2) Score (Best Model): {r2}')

Best Hyperparameters: {'max_depth': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}
R-squared (R2) Score (Best Model): 0.40597273009152074


In [57]:
from sklearn.svm import SVR

In [58]:
model = SVR()

# Train the model
models = []
for col in y_train.columns:
    model = SVR()
    model.fit(X_train, y_train[col])
    models.append(model)

# Make predictions on the test set
y_preds = [model.predict(X_test) for model in models]

# Stack predictions horizontally to form a matrix
y_pred = np.column_stack(y_preds)

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f'R-squared (R2) Score: {r2}')

R-squared (R2) Score: 0.47022255826183773
